In [1]:

import os
import pickle
import sys
from memory_profiler import profile
import pandas as pd
import numpy as np
sys.path.append('/home/ak/Documents/Research/PaperCode/MultiKernelLearning/')
sys.path.append('/home/ak/Documents/Research/PaperCode/singlekernelclf/')
import mkl_data_processing as mkldp

from collections import OrderedDict, defaultdict

In [2]:
import jsonpickle
from MKLpy.preprocessing import normalization, rescale_01
from MKLpy.metrics import pairwise
from MKLpy.algorithms import AverageMKL, EasyMKL
from sklearn.svm import SVC
from collections import OrderedDict
import os
from sklearn.metrics import accuracy_score
import torch
from MKLpy.model_selection import cross_val_score
from MKLpy.multiclass import OneVsRestMKLClassifier, OneVsOneMKLClassifier

from MKLpy.algorithms import AverageMKL, EasyMKL, \
    KOMD  # KOMD is not a MKL algorithm but a simple kernel machine like the SVM

from fileutils import new_feature_utils as nfu
from fileutils.new_feature_utils import CreateMarketFeatures
import multiprocessing

In [3]:

# ------ functions ------- #
def open_pickle_filepath(pickle_file):
    pickle_to_file = pickle.load(open(pickle_file, "rb"), encoding='latin1')

    return pickle_to_file


def forward_Dates(list_of_keys, current_date):
    """
    return all the forward looking dates for each idxKey we use for training

    :param list_of_keys: dates i have model dates for out of sample
    :param current_date: current model date
    :return: forward dates for applying the fitted model
    """
    lookAheadKeys = sorted(i for i in list_of_keys if i > current_date)
    return lookAheadKeys


# ------- paths --------- #

mainPath = mkldp.paths('main')

symbols = sorted(os.listdir(mkldp.paths('symbols_features')))
jointFeatureLocation = os.path.join(mainPath, "ExperimentCommonLocs/JointLocationsDicts")
commonLocs = os.path.join(mainPath, "ExperimentCommonLocs/")

mklOOSPredictionPath = os.path.join(mainPath, "ExperimentCommonLocs/MKLOOSPredictions")
fittedModelsPath = os.path.join(mainPath, "ExperimentCommonLocs/FittedModels")
oosPredictionsPath = os.path.join(mainPath, "ExperimentCommonLocs/OOSPredictions")


In [4]:

# ---------- files

allFiles = os.listdir(jointFeatureLocation) # all files in the joint location area

labelsAlternateLocs = {f:os.path.join(commonLocs, f) for f in os.listdir(os.path.join(mainPath, "ExperimentCommonLocs/")) if str("LabelsAlternate") in f }


alternate_labels_nos = {1:"LabelsAlternateOne", 2:"LabelsAlternateTwo", 3:"LabelsAlternateThree",
                        4:"LabelsAlternateFour", 5:"LabelsAlternateFive", 6:"LabelsAlternateSix"}
# we have 6
# alternative data types

In [5]:
symbolIdx = 3

symbol = symbols[symbolIdx]
print(symbol)

# isolate all the symbol Files
symbolFiles = [f for f in allFiles if str(symbol) in f]


AZN.L


In [6]:

for fileIdx, _ in enumerate(symbolFiles):
    # pick a specific file out of all the symbol files

    file = symbolFiles[fileIdx] # this is the specific file

    select_file_path = os.path.join(jointFeatureLocation, file)  # formulate the path for the specific file
    select_hmm_date = select_file_path.split("_")[3]  # pull out the hmm_date - strip it out
    select_feature_label_date = select_file_path.split("_")[6]  # pull out the label_feature_date from the
    # specific file

    select_label_idx = select_file_path.split("_")[9]  # pull out the label _idx

    print('You are working on symbol:----->', file.split("_")[0],
          'with hmm date: ', select_hmm_date , ' and label feature date' , select_feature_label_date,' and label idx:', select_label_idx)

    unpickled_select_file = open_pickle_filepath(select_file_path)
    # unplickle the select file - and basically this has locations in it for features and labels


You are working on symbol:-----> AZN.L with hmm date:  20170802  and label feature date 20170929  and label idx: 3
You are working on symbol:-----> AZN.L with hmm date:  20170824  and label feature date 20170929  and label idx: 5
You are working on symbol:-----> AZN.L with hmm date:  20170928  and label feature date 20170929  and label idx: 2
You are working on symbol:-----> AZN.L with hmm date:  20170704  and label feature date 20170929  and label idx: 7
You are working on symbol:-----> AZN.L with hmm date:  20170131  and label feature date 20170929  and label idx: 2
You are working on symbol:-----> AZN.L with hmm date:  20170824  and label feature date 20170929  and label idx: 2
You are working on symbol:-----> AZN.L with hmm date:  20170830  and label feature date 20170929  and label idx: 1
You are working on symbol:-----> AZN.L with hmm date:  20170907  and label feature date 20170929  and label idx: 4
You are working on symbol:-----> AZN.L with hmm date:  20170829  and label featu

You are working on symbol:-----> AZN.L with hmm date:  20170929  and label feature date 20170929  and label idx: 4
You are working on symbol:-----> AZN.L with hmm date:  20170803  and label feature date 20170929  and label idx: 4
You are working on symbol:-----> AZN.L with hmm date:  20170817  and label feature date 20170929  and label idx: 4
You are working on symbol:-----> AZN.L with hmm date:  20170915  and label feature date 20170929  and label idx: 4
You are working on symbol:-----> AZN.L with hmm date:  20170822  and label feature date 20170929  and label idx: 3
You are working on symbol:-----> AZN.L with hmm date:  20170818  and label feature date 20170929  and label idx: 7
You are working on symbol:-----> AZN.L with hmm date:  20170124  and label feature date 20170929  and label idx: 5
You are working on symbol:-----> AZN.L with hmm date:  20170810  and label feature date 20170929  and label idx: 4
You are working on symbol:-----> AZN.L with hmm date:  20170928  and label featu

You are working on symbol:-----> AZN.L with hmm date:  20170803  and label feature date 20170929  and label idx: 5
You are working on symbol:-----> AZN.L with hmm date:  20170116  and label feature date 20170929  and label idx: 3
You are working on symbol:-----> AZN.L with hmm date:  20170815  and label feature date 20170929  and label idx: 1
You are working on symbol:-----> AZN.L with hmm date:  20170928  and label feature date 20170929  and label idx: 6
You are working on symbol:-----> AZN.L with hmm date:  20170124  and label feature date 20170929  and label idx: 1
You are working on symbol:-----> AZN.L with hmm date:  20170918  and label feature date 20170929  and label idx: 4
You are working on symbol:-----> AZN.L with hmm date:  20170928  and label feature date 20170929  and label idx: 3
You are working on symbol:-----> AZN.L with hmm date:  20170817  and label feature date 20170929  and label idx: 5
You are working on symbol:-----> AZN.L with hmm date:  20170831  and label featu

In [7]:

hmm_keys = sorted(list(unpickled_select_file.keys()))
# hmm keys - essentially for each of these dates an HMM model was fitted and then it was used out of sample for
# all the other dates
check_dict = defaultdict(dict)
all_good_dict = defaultdict(dict)

for hmm_date_key in hmm_keys:  # pick and hmm date
    feature_label_keys = sorted(
        unpickled_select_file[hmm_date_key].keys())  # each key here unlocks a feature and label set

    for feature_label_date in feature_label_keys:  # make a list of all the feature dates
        features_file_path = unpickled_select_file[hmm_date_key][feature_label_date][
            0]  # this is the feature path
        labels_file_path = unpickled_select_file[hmm_date_key][feature_label_date][1]  # this is the labels path

        if (os.path.isfile(features_file_path) == False) or (os.path.isfile(labels_file_path) == False):
            print('problem here')
        else:
            print('ok----->', feature_label_date)  # if you got to this point we have data so we can mov eon
            labels = pd.read_csv(labels_file_path)  # open labels file
            label_name = str(labels.columns[labels.columns.str.contains(pat='label')].values[0])
            features = open_pickle_filepath(features_file_path)  # opens features file
            hmm_features = nfu.hmm_features_df(features)  # get the hmm features out, so unpack the tuples!
            print('loaded features and labels ')
            if hmm_features.isnull().values.all():  # checking that the HMM features are actually not null
                print('null')
            else:  # if features not null then start moving on!
                market_features_df = CreateMarketFeatures(
                    CreateMarketFeatures(
                        CreateMarketFeatures(df=CreateMarketFeatures(df=labels).ma_spread_duration())
                            .ma_spread()).chaikin_mf()).obv_calc()  # market features dataframe

                df_concat = pd.DataFrame(
                    pd.concat([hmm_features, market_features_df], axis=1, sort='False').dropna())

                df = df_concat[df_concat[label_name].notna()]
                df_final = df.drop(
                    columns=['TradedPrice', 'Duration', 'TradedTime', 'ReturnTradedPrice', 'Volume',
                             label_name])
                y_train = df[df.columns[df.columns.str.contains(pat='label')]].iloc[:, 0]  # training labels
                if df_final.shape[0] < 10:  # make sure it all looks reasonable
                    print(' the ratio of classes is too low. try another label permutation')
                    continue
                else:

                    print("starting model fit")
#                     # put the features in a tensor format
                    X = rescale_01(np.asarray(df_final.values))  # need this for torch
                    Xtr = normalization((torch.Tensor(X)))  # features in a tensor format



problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here
problem here

TypeError: expected Float (got Double)

In [13]:
X = rescale_01(np.asarray(df_final.values)).float()
X

tensor([[0.6285, 0.0017, 0.5351,  ..., 0.4861, 0.8553, 0.9790],
        [0.6290, 0.0013, 0.5336,  ..., 0.4799, 0.8758, 0.9791],
        [0.6270, 0.0000, 0.5320,  ..., 0.4758, 0.8224, 0.9791],
        ...,
        [0.8050, 0.9974, 0.9512,  ..., 0.5516, 0.9970, 0.9788],
        [0.8071, 0.9974, 0.9491,  ..., 0.5821, 0.9971, 0.9761],
        [0.8102, 0.9973, 0.9470,  ..., 0.5210, 0.9965, 0.9753]])

In [14]:
Xtr = normalization((torch.Tensor(X)))

In [15]:
Ytr = torch.Tensor(y_train.values)  # put the labels in a tensor format

In [16]:
from MKLpy.metrics import pairwise
from MKLpy import generators
from MKLpy.algorithms import AverageMKL, EasyMKL
from sklearn.svm import SVC

In [24]:
KLrbf = generators.RBF_generator(Xtr, gamma=[.01, .1, .25,
                                                                         .5, .9, 1.25])  # get a few RBF Kernels ready - maybe need more here
print('done with kernel')
best_results = {}

C_range = [0.1, 1]
lam_range = [0.2]


for C_choice in C_range:
    base_learner = SVC(C=C_choice)  # "hard"-margin svm
    clf = EasyMKL(lam=0.2, multiclass_strategy='ova', learner=base_learner).fit(KLrbf,Ytr)
#     print('(%d vs all): ' % sol,
#           clf.solution[
#               sol].weights)  # need to store these results somewhere

# #     for lam in lam_range:  # possible lambda values for the EasyMKL algorithm
# #         # MKLpy.model_selection.cross_val_score performs the cross validation automatically, it may returns
# #         # accuracy, auc, or F1 scores
# #         scores = cross_val_score(KLrbf, Ytr, EasyMKL(learner=base_learner, lam=lam),
# #                                  n_folds=5,
# #                                  scoring='accuracy')  # get the cross-validation scores
# #         acc = np.mean(scores)
# #         if not best_results or best_results['score'] < acc:
# #             best_results = {'C': C_choice, 'lam': lam, 'score': acc,
# #                             'scores': scores}  # these should get dumped somewhere



done with kernel


In [ ]:
#     print('done')
#     best_learner = SVC(C=best_results['C'])
#     clf = EasyMKL(learner=best_learner, lam=best_results['lam']).fit(KLrbf, Ytr)
#     y_pred = clf.predict(KLrbf)
#     accuracy = accuracy_score(Ytr, y_pred)
#     print('accuracy on the test set: %.3f, with lambda=%.2f' % (
#         accuracy, best_results['lam']))
#     print(scores)